In [233]:
import pandas as pd
import numpy as np

# Default configuration for time periods in traffic data
input_file = "outputs/2026-01-30/PCE0.75/model_run.csv"
run_folder = 'consecutive_increase'

# Here we load th value of the counts and we multiply the peak hour values by a constant
lights_w = 1

heavies_w = 3
heavies_w_toll = 3
heavies_w_vot = 3

medium_A_w = 2.75 # 2.5 # TBD: Maybe try 2.5 or 2.75 for every pce value
medium_A_w_toll = 4
medium_A_w_vot = 3

medium_B_w = 2.75 # 2.5
medium_B_w_toll = 4
medium_B_w_vot = 3

heavy_A_w = 3
heavy_A_w_toll = 7
heavy_A_w_vot = 5

heavy_B_w = 3
heavy_B_w_toll = 3
heavy_B_w_vot = 3

# Default time periods list (for reference)
default_time_periods = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

# Create the base scenario: hour -> time period mapping
hour_to_period = {
    0: "Night",
    1: "Night",
    2: "Night",
    3: "Night",
    4: "Night",
    5: "Night",
    6: "AM-Early",
    7: "AM-Peak",
    8: "AM-Peak",
    9: "AM-Shoulder",
    10: "MD",
    11: "MD",
    12: "MD",
    13: "MD",
    14: "MD",
    15: "PM-Shoulder",
    16: "PM-Peak",
    17: "PM-Peak",
    18: "PM-Peak",
    19: "PM-Late",
    20: "PM-Late",
    21: "PM-Late",
    22: "Night",
    23: "Night"
}

period_to_period = {
    'Evening': 'Night',
    'Evening': 'PM-Late',
    'EarlyAM': 'AM-Early',
    'AM': 'AM-Peak',
    'AM': 'AM-Shoulder',
    'Midday': 'MD',
    'Midday': 'PM-Shoulder',
    'PM': 'PM-Peak'
}

# Define the segments and their parameters

awt_adt = 1.1 # Average weekday traffic (AWT) to average daily traffic (ADT) ratio
peak_factor = 1 # 1.05 # Peak factor for adjustment at peak hour traffic

hov_percentage = pd.DataFrame({
    'Year' : [2025,2032,2040,2050],
    'HOV percentage' : [0,0,0,0]
})

hov_percentage.set_index('Year', inplace=True)

# Define segment parameters base
seg_params = pd.DataFrame({
    'SegDir':   ["1NB","1SB","2NB","2SB","3NB","3SB","4NB","4SB","5NB","5SB","6NB","6SB","7NB","7SB","8NB","8SB","9NB","9SB","10NB","10SB"],
    'Length':    [1.3,1.3,1.3,1.3,0.5,0.5,1.6,1.6,2,2,3.6,3.6,2.9,2.9,3.8,3.8,3.4,3.4,4.5,4.5],
    'Inscope':   [0.94,0.94,1,1,1,1,1,1,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94,0.94], # [0.82,0.82,0.92,0.92,0.88,0.88,0.88,0.88,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8],  
    'Lanes_GP':  [4]*20, #
    'Lanes_ML':  [2]*20, # Lanes_ML': [2,2,2,2,2,2,2,2,3,3,2,2,2,2], # Do test changing segment 5
    'CapPerLane_GP': [2000]*20,
    'CapPerLane_ML': [1800]*20,
    'Speed_GP':  [55]*6 + [65]*2 + [70]*12,
    'Speed_ML':  [60]*6 + [70]*2 + [70]*12,
    'Alpha_GP':  [1]*20,
    'Beta_GP':   [6]*20,
    'Alpha_ML':  [1.6]*20,
    'Beta_ML':   [6.3]*20,
    'Min_Toll_2016': [None]*20,
    'Max_Toll_2016': [None]*20,
    'LanesGP_AM_Peak': [5]*20,
    'LanesGP_PM_Peak': [5]*20,
})

seg_params.set_index('SegDir', inplace=True)

# Compute capacities as lanes * cap per lane
seg_params['Cap_GP'] = seg_params['Lanes_GP'] * seg_params['CapPerLane_GP']
seg_params['Cap_ML'] = seg_params['Lanes_ML'] * seg_params['CapPerLane_ML']

# Compute peak capacities as Alpha * base capacity
seg_params['CapGP_Peak'] = seg_params['Alpha_GP'] * seg_params['Cap_GP']
seg_params['CapML_Peak'] = seg_params['Alpha_ML'] * seg_params['Cap_ML']

# Optional: if you want integer capacities
seg_params[['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']] = seg_params[
    ['Cap_GP','Cap_ML','CapGP_Peak','CapML_Peak']
].astype(int)

# Preview
seg_params

,Length,Inscope,Lanes_GP,Lanes_ML,CapPerLane_GP,CapPerLane_ML,Speed_GP,Speed_ML,Alpha_GP,Beta_GP,Alpha_ML,Beta_ML,Min_Toll_2016,Max_Toll_2016,LanesGP_AM_Peak,LanesGP_PM_Peak,Cap_GP,Cap_ML,CapGP_Peak,CapML_Peak
SegDir,,,,,,,,,,,,,,,,,,,,
1NB,1.3,0.94,4,2,2000,1800,55,60,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
1SB,1.3,0.94,4,2,2000,1800,55,60,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
2NB,1.3,1.00,4,2,2000,1800,55,60,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
2SB,1.3,1.00,4,2,2000,1800,55,60,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
3NB,0.5,1.00,4,2,2000,1800,55,60,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
3SB,0.5,1.00,4,2,2000,1800,55,60,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
4NB,1.6,1.00,4,2,2000,1800,65,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
4SB,1.6,1.00,4,2,2000,1800,65,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760
5NB,2.0,0.94,4,2,2000,1800,70,70,1,6,1.6,6.3,None,None,5,5,8000,3600,8000,5760


In [234]:
def get_vot(row):

    max_VC = 1.2  # TBD: check if we need to change this value
    ETC_discount = 0.15
    
    captureRateLights =  row["CaptureRateLights"]
    captureRateMediumA =  row["CaptureRateMediumA"]
    captureRateMediumB =  row["CaptureRateMediumB"]
    captureRateHeavyA =  row["CaptureRateHeavyA"]

    tollLights = row["TollLights"]
    tollMediumA = row["TollMediumA"]
    tollMediumB = row["TollMediumB"]
    tollHeavyA = row["TollHeavyA"]

    # ml_pce = seg_params.loc[row["SegDir"], 'Inscope'] * (
    #     row["TotalLights"] * lights_w * captureRateLights + 
    #     row["TotalMediumA"] * medium_A_w * captureRateMediumA +
    #     row["TotalMediumB"] * medium_B_w * captureRateMediumB +
    #     row["TotalHeavyA"] * heavy_A_w * captureRateHeavyA
    # )

    ml_pce = (
        row["InScopeLights"] * lights_w * captureRateLights + 
        row["InScopeMediumA"] * medium_A_w * captureRateMediumA +
        row["InScopeMediumB"] * medium_B_w * captureRateMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureRateHeavyA
    )
    
    gp_pce = np.min(np.array([row["Corridor PCE"] - ml_pce, row["Max VC"] * seg_params['Cap_GP'][row['SegDir']]]))
    
    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    timeML = 60 * row["Length"] / speedML

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    timeSavings = timeML - timeGP

    gp_vc = gp_pce / row["Capacity GP"]

    ml_vc = (ml_pce + row["HOV3"]) / row["Capacity ML"]

    # bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"])) 

    # bonusAux = (2.5 if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else 1.6)

    bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"]))

    bonusAux = (row["BonusPeak"] if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else row["BonusOffPeak"]) 

    bonusSeg = ((bonusAux if gp_vc > 0.3 else 0) * (gp_vc - ml_vc)) * row["Length"] # TBD: add condition for gp_vc

    if speedGP > speedML:
        bonusSeg = 0
        bonusRel = 0

    votLights = -60 * (1 - ETC_discount) * row["Length"] * tollLights / (timeSavings - bonusSeg - bonusRel) # TBD: add household income effect

    votMediumA = -60 * (1 - ETC_discount) * row["Length"] * tollMediumA / (timeSavings - bonusSeg - bonusRel)

    votMediumB = -60 * (1 - ETC_discount) * row["Length"] * tollMediumB / (timeSavings - bonusSeg - bonusRel)

    votHeavyA = -60 * (1 - ETC_discount) * row["Length"] * tollHeavyA / (timeSavings - bonusSeg - bonusRel)

    return pd.Series([votLights, votMediumA, votMediumB, votHeavyA, speedGP, speedML, bonusSeg, timeML, timeGP, timeSavings, bonusRel], index=["VOT Lights", "VOT MediumA", "VOT MediumB", "VOT HeavyA", "Speed GP", "Speed ML", "Bonus Seg", "Time ML", "Time GP", "Time Savings", "BonusRel"])

In [235]:
from scipy.stats import lognorm
from scipy.optimize import minimize, NonlinearConstraint, Bounds

ETC_discount = 0.15
max_VC = 1.2  # TBD: check if we need to change this value

def objective_integrated(x, row, M):

    captureRateLights, captureRateMediumA, captureRateMediumB, captureRateHeavyA, tollLights = x

    ml_pce = (
        row["InScopeLights"] * lights_w * captureRateLights + 
        row["InScopeMediumA"] * medium_A_w * captureRateMediumA +
        row["InScopeMediumB"] * medium_B_w * captureRateMediumB +
        row["InScopeHeavyA"] * heavy_A_w * captureRateHeavyA
    )

    share_coef = 0

    share_coef = 0.1 * np.max([0, tollLights - 2 * row["MaxToll"]]) + 0.25 * np.max([0, np.min([tollLights - 3/2 * row["MaxToll"], 2 * row["MaxToll"] - 3/2 * row["MaxToll"] ])]) + 0.5 * np.max([0, np.min([tollLights - row["MaxToll"], 3/2 * row["MaxToll"] - row["MaxToll"] ])])

    tollMediumA = tollLights * medium_A_w_toll
    tollMediumB = tollLights * medium_B_w_toll
    tollHeavyA = tollLights * heavy_A_w_toll

    gp_pce = np.min(np.array([row["Corridor PCE"] - ml_pce, row["Max VC"] * seg_params['Cap_GP'][row['SegDir']]]))
    
    speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests

    timeML = 60 * row["Length"] / speedML

    speedGP =  row["Speed GP"] / (1 + row["Alpha GP"] * ((gp_pce / row["Capacity GP"]) ** row["Beta GP"]))

    timeGP = 60 * row["Length"] / speedGP

    timeSavings = timeML - timeGP

    trigger_speed = 0
    if speedGP > speedML:
        trigger_speed = 1000000000

    gp_vc = gp_pce / row["Capacity GP"]

    ml_vc = (ml_pce + row["HOV3"]) / row["Capacity ML"]

    bonusRel = 0.7 * 60 * row["Length"] * ((1/speedGP)-(1/row["Speed GP"]))

    bonusAux = (row["BonusPeak"] if (row["Period"] == "AM-Peak" or row["Period"] == "PM-Peak") else row["BonusOffPeak"])
    
    bonusSeg = ((bonusAux if gp_vc > 0.3 else 0) * (gp_vc - ml_vc)) * row["Length"] # TBD: add condition for gp_vc

    if speedGP > speedML:
        bonusSeg = 0
        bonusRel = 0

    votLights = -60 * (1 - ETC_discount) * row["Length"] * tollLights / (timeSavings - bonusSeg - bonusRel) # TBD: add household income effect

    votMediumA = -60 * (1 - ETC_discount) * row["Length"] * tollMediumA / (timeSavings - bonusSeg - bonusRel)

    votMediumB = -60 * (1 - ETC_discount) * row["Length"] * tollMediumB / (timeSavings - bonusSeg - bonusRel)

    votHeavyA = -60 * (1 - ETC_discount) * row["Length"] * tollHeavyA / (timeSavings - bonusSeg - bonusRel)

    # TBD: Integrate Reliability
    # Here we compute the lognormal cumulative function for the light vehicles

    mu_lights = row["B1"]

    mu_heavies = mu_lights

    std_dev = row["B2"]

    scale_lights = np.exp(mu_lights)

    scale_heavies = np.exp(mu_heavies) * heavies_w_vot # TBD: change heavies betas (ask Borja)

    scale_medium_A = np.exp(mu_heavies) * medium_A_w_vot

    scale_medium_B = np.exp(mu_heavies) * medium_B_w_vot #

    scale_heavy_A = np.exp(mu_heavies) * heavy_A_w_vot

    # We create a lognormal distribution object

    dist_lights = lognorm(s=std_dev, scale=scale_lights)

    dist_heavies = lognorm(s=std_dev, scale=scale_heavies)

    dist_medium_A = lognorm(s=std_dev, scale=scale_medium_A)

    dist_medium_B = lognorm(s=std_dev, scale=scale_medium_B)

    dist_heavy_A = lognorm(s=std_dev, scale=scale_heavy_A)

    if votLights > 0:

        calcCaptureRateLights = 1 - dist_lights.cdf(votLights)

        calcCaptureMediumA = 1 - dist_medium_A.cdf(votMediumA)

        calcCaptureMediumB = 1 - dist_medium_B.cdf(votMediumB)

        calcCaptureHeavyA = 1 - dist_heavy_A.cdf(votHeavyA)

    else:

        calcCaptureRateLights = 0

        calcCaptureMediumA = 0

        calcCaptureMediumB = 0

        calcCaptureHeavyA = 0

    convergenceLights = calcCaptureRateLights - captureRateLights

    convergenceMediumA = calcCaptureMediumA - captureRateMediumA

    convergenceMediumB = calcCaptureMediumB - captureRateMediumB

    convergenceHeavyA = calcCaptureHeavyA - captureRateHeavyA

    tollBase = np.min([tollLights, row["MaxToll"]])

    shareLights = tollBase + share_coef
    shareMediumA = shareLights * medium_A_w_toll
    shareMediumB = shareLights * medium_B_w_toll
    shareHeavyA = shareLights * heavy_A_w_toll


    obj = (
        - shareLights * calcCaptureRateLights * row["Length"] * row["InScopeLights"]
        - shareMediumA * calcCaptureMediumA * row["Length"] * row["InScopeMediumA"]
        - shareMediumB * calcCaptureMediumB * row["Length"] * row["InScopeMediumB"]
        - shareHeavyA * calcCaptureHeavyA * row["Length"] * row["InScopeHeavyA"]
        + M * convergenceLights ** 2
        + M * convergenceMediumA ** 2
        + M * convergenceMediumB ** 2
        + M * convergenceHeavyA ** 2
        + M * trigger_speed
    )

    return np.nan_to_num(obj, nan=1e9, posinf=1e9, neginf=1e9)

In [236]:
first_model_df = pd.read_csv(f"{input_file}")

previous_seg_dict = {
    "1SB": ["3SB"],
    "2SB": ["3SB"],
    "3SB": ["4SB"],
    "4SB": ["5SB"],
    "5SB": ["6SB"],
    "6SB": ["7SB"],
    "7SB": ["8SB"],
    "8SB": ["9SB"],
    "9SB": ["10SB"],
    "10SB": [],
    "10NB": ["9NB"],
    "9NB": ["8NB"],
    "8NB": ["7NB"],
    "7NB": ["6NB"],
    "6NB": ["5NB"],
    "5NB": ["4NB"],
    "4NB": ["3NB"],
    "3NB": ["1NB", "2NB"],
    "2NB": [],
    "1NB": [],
}

first_model_df["Previous Seg"] = first_model_df["SegDir"].map(previous_seg_dict)

In [237]:
import pandas as pd
import sys

# first_model_df = first_model_df[(first_model_df["Year"] == 2050) or ()].reset_index()

def optimize_row(row):

    M = 10000000 # Best value of M so far

    # We get the value of the toll for the highest value of revenue

    captureLights = row["CaptureRateLights"] # / seg_params.loc[row["SegDir"], 'Inscope']

    captureMediumA = row["CaptureRateMediumA"] # / seg_params.loc[row["SegDir"], 'Inscope']

    captureMediumB = row["CaptureRateMediumB"] # / seg_params.loc[row["SegDir"], 'Inscope']

    captureHeavyA = row["CaptureRateHeavyA"] # / seg_params.loc[row["SegDir"], 'Inscope']

    toll = row["TollLights"]

    max_capture = row["MaxCapture"]

    capture_first_guess = row["MaxCapture"]

    min_toll = row["MinToll"]

    for segdir in row["Previous Seg"]:

        row_aux = first_model_df[(first_model_df["SegDir"] == segdir) & (first_model_df["Year"] == row["Year"]) & (first_model_df["Period"] == row["Period"])].iloc[0]

        if toll < row["Soft Cap"] and row_aux["TollLights"] > row["Soft Cap"]:
  
            cap_constraint = row_aux["TollLights"]

            ub = [
                max_capture,
                max_capture,
                max_capture,
                max_capture,
                cap_constraint
            ]

            lb = [
                row["MinCapture"],
                row["MinCapture"],
                row["MinCapture"],
                row["MinCapture"],
                min_toll
            ]

            bounds = Bounds(lb=lb, ub=ub)

            x0 = [
                capture_first_guess * 0.5,
                capture_first_guess * 0.5,
                capture_first_guess * 0.5,
                capture_first_guess * 0.5,
                (cap_constraint + min_toll) / 2
            ] # Initial guess, TBD

            Max_iter = 10000

            result = minimize(
                objective_integrated,
                x0,
                method='trust-constr',
                args=(row,M,),
                bounds=bounds,
                # constraints=[nlc],
                options={
                    "verbose": 0,
                    "maxiter": Max_iter,
                    "gtol": 1e-6,
                    "xtol": 1e-6,
                    "barrier_tol": 1e-6,
                    "initial_tr_radius": 1.0,
                    "initial_constr_penalty": 1.0,
                    "sparse_jacobian": True
                }
            )

            if not result.success:
                x0_new = np.clip(result.x, bounds.lb, bounds.ub)  # project back into bounds
                result = minimize(
                    objective_integrated,
                    x0_new,
                    method="trust-constr",
                    args=(row,M,),
                    bounds=bounds,
                    # constraints=[nlc],
                    options={"maxiter": 200000}
                )

            x_opt = result.x
            z_opt = result.fun
            
            x_opt = np.minimum(bounds.ub, np.maximum(bounds.lb, result.x))

            if not result.success:
                print(x_opt)

            # x_opt = check_pce(row, x_opt)

            # if result.success:
            #     x_opt = result.x
            #     z_opt = result.fun
            # else:
            #     x_opt, z_opt = np.nan, np.nan            

            return pd.Series([x_opt[0], x_opt[1], x_opt[2], x_opt[3], x_opt[4]], 
                        index=["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", 
                            "CaptureRateHeavyA", "TollLights"])


    # print(f'Segment: {row["SegDir"]}, Period: {row["Period"]}, toll: {toll}, traffic: {traffic_lane}, speed: {speedML}')

    return pd.Series([captureLights, captureMediumA, captureMediumB, captureHeavyA, toll], 
                index=["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", "CaptureRateHeavyA", "TollLights"])

first_model_df[["CaptureRateLights", "CaptureRateMediumA", "CaptureRateMediumB", "CaptureRateHeavyA", "TollLights"]] = first_model_df.apply(
    optimize_row, axis=1, result_type='expand'
)

C:\Users\crisp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\scipy\optimize\_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\3163415696.py:28: RuntimeWarning: invalid value encountered in scalar power
  speedML = row["Speed ML"] / (1 + row["Alpha ML"] * (((ml_pce + row["HOV3"]) / row["Capacity ML"])** row["Beta ML"])) # TBD: maintain beta ML for future tests


In [238]:
def get_share(row):
    
    devShare = 0

    tdotShare = 0

    baseToll = row["TollLights"]

    share4x = np.max([0, baseToll - 2 * row["MaxToll"]])
    share3x = np.max([0, np.min([baseToll - 3/2 * row["MaxToll"], 2 * row["MaxToll"] - 3/2 * row["MaxToll"] ])])
    share2x = np.max([0, np.min([baseToll - row["MaxToll"], 3/2 * row["MaxToll"] - row["MaxToll"] ])])

    devShare = 0.1 * share4x + 0.25 * share3x + 0.5 * share2x
    devShare2x = 0.5 * share2x
    devShare3x = 0.25 * share3x
    devShare4x = 0.1 * share4x

    tdotShare = 0.9 * share4x + 0.75 * share3x + 0.5 * share2x
    tdotShare2x = 0.5 * share2x
    tdotShare3x = 0.75 * share3x
    tdotShare4x = 0.9 * share4x

    if row["TollLights"] > row["MaxToll"]:
        baseToll = row["MaxToll"]

    return pd.Series([baseToll, devShare, tdotShare, devShare2x, devShare3x, devShare4x, tdotShare2x, tdotShare3x, tdotShare4x],
                     index=["baseToll", "devTollShare", "tdotTollShare", "devTollShare2x", "devTollShare3x", "devTollShare4x", "tdotTollShare2x","tdotTollShare3x","tdotTollShare4x"])

In [239]:
first_model_df[["baseToll","devTollShare","tdotTollShare","devTollShare2x","devTollShare3x","devTollShare4x", "tdotTollShare2x","tdotTollShare3x","tdotTollShare4x"]] = first_model_df.apply(
    get_share, axis=1, result_type='expand'
)

first_model_df["TollMediumA"] = first_model_df.apply(
    lambda row: row["TollLights"] * medium_A_w_toll,
    axis=1
)

first_model_df["TollMediumB"] = first_model_df.apply(
    lambda row: row["TollLights"] * medium_B_w_toll,
    axis=1
)

first_model_df["TollHeavyA"] = first_model_df.apply(
    lambda row: row["TollLights"] * heavy_A_w_toll,
    axis=1
)

bands = ["", "2x", "3x", "4x"]

for band in bands:

    first_model_df[f"devTollMediumA{band}"] = first_model_df.apply(
        lambda row: row[f"devTollShare{band}"] * medium_A_w_toll,
        axis=1
    )

    first_model_df[f"devTollMediumB{band}"] = first_model_df.apply(
        lambda row: row[f"devTollShare{band}"] * medium_B_w_toll,
        axis=1
    )

    first_model_df[f"devTollHeavyA{band}"] = first_model_df.apply(
        lambda row: row[f"devTollShare{band}"] * heavy_A_w_toll,
        axis=1
    )

    first_model_df[f"tdotTollMediumA{band}"] = first_model_df.apply(
        lambda row: row[f"tdotTollShare{band}"] * medium_A_w_toll,
        axis=1
    )

    first_model_df[f"tdotTollMediumB{band}"] = first_model_df.apply(
        lambda row: row[f"tdotTollShare{band}"] * medium_B_w_toll,
        axis=1
    )

    first_model_df[f"tdotTollHeavyA{band}"] = first_model_df.apply(
        lambda row: row[f"tdotTollShare{band}"] * heavy_A_w_toll,
        axis=1
    )

first_model_df["baseTollMediumA"] = first_model_df.apply(
    lambda row: row["baseToll"] * medium_A_w_toll,
    axis=1
)

first_model_df["baseTollMediumB"] = first_model_df.apply(
    lambda row: row["baseToll"] * medium_B_w_toll,
    axis=1
)

first_model_df["baseTollHeavyA"] = first_model_df.apply(
    lambda row: row["baseToll"] * heavy_A_w_toll,
    axis=1
)

first_model_df[["VOT Lights", "VOT MediumA", "VOT MediumB", "VOT HeavyA", "Speed GP Real", "Speed ML Real", "Bonus Seg", "Time ML", "Time GP", "Time Savings", "BonusRel"]] = first_model_df.apply(
    get_vot, axis=1, result_type='expand'
)

first_model_df

,Unnamed: 0,index,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,...,TransactionsHeavyALength,TotalVeh_hours,GPDayLength_Lights,GPDayLength_MediumA,GPDayLength_MediumB,GPDayLength_HeavyA,GPDayLength_HeavyB,Difference in Travel Time,Implied Min VOT,Previous Seg
0,0,0,2025,1NB,1,NB,Night,7,OP,NT,...,21.510637,12054.000000,12445.123388,333.271504,90.078683,2235.289363,273.000000,-0.083882,131.058601,[]
1,1,1,2025,1NB,1,NB,AM-Early,1,OP,AM,...,16.460114,5777.000000,6057.245003,185.201770,96.893038,317.639886,15.600000,-0.116687,361.953811,[]
2,2,2,2025,1NB,1,NB,AM-Peak,3,Peak,AM,...,202.796364,20535.511652,18446.524846,810.834443,200.805174,1661.231977,43.148804,-0.381529,154.208771,[]
3,3,3,2025,1NB,1,NB,AM-Shoulder,1,OP,AM,...,83.456890,6282.794226,5478.465611,271.942362,75.473147,727.753832,24.376950,-0.353598,137.596336,[]
4,4,4,2025,1NB,1,NB,MD,5,OP,MD,...,330.848489,21465.000000,17967.727602,730.089949,199.114977,2139.151511,71.500000,-0.237947,153.493932,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,315,2032,10SB,10,SB,AM-Shoulder,1,OP,AM,...,188.029110,4121.101809,12452.244841,665.792801,289.599103,2107.584641,84.172504,-0.045132,1184.683855,[]
316,316,316,2032,10SB,10,SB,MD,5,OP,MD,...,758.239406,27933.000000,92341.140585,3652.293435,1270.182798,10829.260594,364.500000,-0.042698,1255.285180,[]
317,317,317,2032,10SB,10,SB,PM-Shoulder,1,OP,PM,...,80.177513,5023.000000,16988.583431,514.012773,128.502891,1061.922487,50.400000,-0.050432,1054.096237,[]
318,318,318,2032,10SB,10,SB,PM-Peak,3,Peak,PM,...,534.822620,16034.335131,49391.516944,1252.438686,238.497464,4595.928662,236.866735,-0.055850,943.932887,[]


In [240]:
first_model_df["Reliability %"] = first_model_df.apply(
    lambda row: row["BonusRel"] / (row["BonusRel"] + row["Bonus Seg"] - row["Time Savings"]),
    axis=1
)

first_model_df["Bonus %"] = first_model_df.apply(
    lambda row: row["Bonus Seg"] / (row["BonusRel"] + row["Bonus Seg"] - row["Time Savings"]),
    axis=1
)

first_model_df["Time Savings %"] = first_model_df.apply(
    lambda row: - row["Time Savings"] / (row["BonusRel"] + row["Bonus Seg"] - row["Time Savings"]),
    axis=1
)

first_model_df["MLVeh_Lights"] = first_model_df.apply(
    lambda row: row["InScopeLights"] * row["CaptureRateLights"],
    axis=1
)

first_model_df["GPVeh_Lights"] = first_model_df.apply(
    lambda row: row["TotalLights"] - row["MLVeh_Lights"] - row["HOV3"],
    axis=1
)

first_model_df["MLVeh_MediumA"] = first_model_df.apply(
    lambda row: row["InScopeMediumA"] * row["CaptureRateMediumA"],
    axis=1
)

first_model_df["GPVeh_MediumA"] = first_model_df.apply(
    lambda row: row["TotalMediumA"] - row["MLVeh_MediumA"],
    axis=1
)

first_model_df["MLVeh_MediumB"] = first_model_df.apply(
    lambda row: row["InScopeMediumB"] * row["CaptureRateMediumB"],
    axis=1
)

first_model_df["GPVeh_MediumB"] = first_model_df.apply(
    lambda row: row["TotalMediumB"] - row["MLVeh_MediumB"],
    axis=1
)

first_model_df["MLVeh_HeavyA"] = first_model_df.apply(
    lambda row: row["InScopeHeavyA"] * row["CaptureRateHeavyA"],
    axis=1
)

first_model_df["GPVeh_HeavyA"] = first_model_df.apply(
    lambda row: row["TotalHeavyA"] - row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["MLVeh"] = first_model_df.apply(
    lambda row: row["MLVeh_Lights"] + row["MLVeh_MediumA"] + row["MLVeh_MediumB"] + row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["GPVeh"] = first_model_df.apply(
    lambda row: row["GPVeh_Lights"] + row["GPVeh_MediumA"] + row["GPVeh_MediumB"] + row["GPVeh_HeavyA"] + row["TotalHeavyB"],
    axis=1
)

first_model_df["GPVehDay"] = first_model_df.apply(
    lambda row: row["GPVeh"] * row["Hours/Day"],
    axis=1
)

first_model_df["ML PCE"] = first_model_df.apply(
    lambda row:row["MLVeh_Lights"] * lights_w
    + row["MLVeh_MediumA"] * medium_A_w
    + row["MLVeh_MediumB"] * medium_B_w
    + row["MLVeh_HeavyA"] * heavy_A_w, # TBD: is HOV included in the PCE?
    axis=1
)

first_model_df["ML V/C"] = first_model_df.apply(
    lambda row:row["ML PCE"] / row["Capacity ML"],
    axis=1
)

first_model_df["ML Volume"] = first_model_df.apply(
    lambda row: row["ML PCE"] * row["Hours/Day"],
    axis=1
)

first_model_df["ML PCE Total"] = first_model_df.apply(
    lambda row: row["ML PCE"] * row["Hours/Day"] * row["Length"],
    axis=1
)

first_model_df["GP PCE"] = first_model_df.apply(
    lambda row:row["GPVeh_Lights"] * lights_w
    + row["GPVeh_MediumA"] * medium_A_w
    + row["GPVeh_MediumB"] * medium_B_w
    + row["GPVeh_HeavyA"] * heavy_A_w
    + row["TotalHeavyB"] * heavy_B_w, # TBD: is HOV included in the PCE?
    axis=1
)

first_model_df["GP Volume"] = first_model_df.apply(
    lambda row: row["GP PCE"] * row["Hours/Day"],
    axis=1
)

first_model_df["GP PCE Total"] = first_model_df.apply(
    lambda row: row["GP PCE"] * row["Hours/Day"] * row["Length"],
    axis=1
)

# Corridor Vals

first_model_df["Highway PCE"] = first_model_df.apply(
    lambda row: row["ML PCE"] + row["GP PCE"],
    axis=1
)

first_model_df["Highway Volume"] = first_model_df.apply(
    lambda row: row["ML Volume"] + row["GP Volume"],
    axis=1
)

first_model_df["Highway PCE Total"] = first_model_df.apply(
    lambda row: row["ML PCE Total"] + row["GP PCE Total"],
    axis=1
)

first_model_df["Highway V/C"] = first_model_df.apply(
    lambda row: (row["ML PCE"] + row["GP PCE"]) / (row["Capacity GP"]),
    axis=1
)

# We add the TollPerSeg column

first_model_df["TollLightsPerSeg"] = first_model_df.apply(
    lambda row: row["TollLights"] * row["Length"],
    axis=1
)

first_model_df["TollMediumAPerSeg"] = first_model_df.apply(
    lambda row: row["TollMediumA"] * row["Length"],
    axis=1
)

first_model_df["TollMediumBPerSeg"] = first_model_df.apply(
    lambda row: row["TollMediumB"] * row["Length"],
    axis=1
)

first_model_df["TollHeavyAPerSeg"] = first_model_df.apply(
    lambda row: row["TollHeavyA"] * row["Length"],
    axis=1
)

# Toll blend
first_model_df["TollBlend"] = first_model_df.apply(
    lambda row: (row["TollLights"] * row["MLVeh_Lights"] 
                + row["TollMediumA"] * row["MLVeh_MediumA"]
                + row["TollMediumB"] * row["MLVeh_MediumB"]
                + row["TollHeavyA"] * row["MLVeh_HeavyA"]
                ) / (row["MLVeh"]),
    axis=1
)

first_model_df["RevenuePerHour"] = first_model_df.apply(
    lambda row: row["TollLightsPerSeg"] * row["MLVeh_Lights"]
    + row["TollMediumAPerSeg"] * row["MLVeh_MediumA"]
    + row["TollMediumBPerSeg"] * row["MLVeh_MediumB"]
    + row["TollHeavyAPerSeg"] * row["MLVeh_HeavyA"],
    axis=1
)

############### Revenue Stream #######################

for band in bands:

    first_model_df[f"DevRevenuePerDay{band}"] = first_model_df.apply(
        lambda row: (row[f"devTollShare{band}"] * row["MLVeh_Lights"]
        + row[f"devTollMediumA{band}"] * row["MLVeh_MediumA"]
        + row[f"devTollMediumB{band}"] * row["MLVeh_MediumB"]
        + row[f"devTollHeavyA{band}"] * row["MLVeh_HeavyA"]) * row["Length"] * row["Hours/Day"],
        axis=1
    )

    first_model_df[f"TDOTRevenuePerDay{band}"] = first_model_df.apply(
        lambda row: (row[f"tdotTollShare{band}"] * row["MLVeh_Lights"]
        + row[f"tdotTollMediumA{band}"] * row["MLVeh_MediumA"]
        + row[f"tdotTollMediumB{band}"] * row["MLVeh_MediumB"]
        + row[f"tdotTollHeavyA{band}"] * row["MLVeh_HeavyA"]) * row["Length"] * row["Hours/Day"],
        axis=1
    )

first_model_df["BaseRevenuePerDay"] = first_model_df.apply(
    lambda row: (row["baseToll"] * row["MLVeh_Lights"]
    + row["baseTollMediumA"] * row["MLVeh_MediumA"]
    + row["baseTollMediumB"] * row["MLVeh_MediumB"]
    + row["baseTollHeavyA"] * row["MLVeh_HeavyA"]) * row["Length"] * row["Hours/Day"],
    axis=1
)

######################################################

first_model_df["RevenuePerHourLights"] = first_model_df.apply(
    lambda row: row["TollLightsPerSeg"] * row["MLVeh_Lights"],
    axis=1
)

first_model_df["RevenuePerHourMediumA"] = first_model_df.apply(
    lambda row: row["TollMediumAPerSeg"] * row["MLVeh_MediumA"],
    axis=1
)

first_model_df["RevenuePerHourMediumB"] = first_model_df.apply(
    lambda row: row["TollMediumBPerSeg"] * row["MLVeh_MediumB"],
    axis=1
)

first_model_df["RevenuePerHourHeavyA"] = first_model_df.apply(
    lambda row: row["TollHeavyAPerSeg"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["RevenuePerDay"] = first_model_df.apply(
    lambda row: row["RevenuePerHour"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayLights"] = first_model_df.apply(
    lambda row: row["RevenuePerHourLights"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayMediumA"] = first_model_df.apply(
    lambda row: row["RevenuePerHourMediumA"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayMediumB"] = first_model_df.apply(
    lambda row: row["RevenuePerHourMediumB"] * row["Hours/Day"],
    axis=1
)

first_model_df["RevenuePerDayHeavyA"] = first_model_df.apply(
    lambda row: row["RevenuePerHourHeavyA"] * row["Hours/Day"],
    axis=1
)

first_model_df["TransactionsDay_Lights"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_Lights"],
    axis=1
)

first_model_df["TransactionsDay_MediumA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_MediumA"],
    axis=1
)

first_model_df["TransactionsDay_MediumB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_MediumB"],
    axis=1
)

first_model_df["TransactionsDay_HeavyA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["MLVeh_HeavyA"],
    axis=1
)

first_model_df["TransactionsDay"] = first_model_df.apply(
    lambda row: row["TransactionsDay_Lights"] + row["TransactionsDay_MediumA"] + row["TransactionsDay_MediumB"] + row["TransactionsDay_HeavyA"],
    axis=1
)

first_model_df["GPDay_Lights"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_Lights"],
    axis=1
)

first_model_df["GPDay_MediumA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_MediumA"],
    axis=1
)

first_model_df["GPDay_MediumB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_MediumB"],
    axis=1
)

first_model_df["GPDay_HeavyA"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["GPVeh_HeavyA"],
    axis=1
)

first_model_df["GPDay_HeavyB"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["TotalHeavyB"],
    axis=1
)


first_model_df["TransactionsLightsLength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_Lights"] * row["Length"],
    axis=1
)

first_model_df["TransactionsMediumALength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_MediumA"] * row["Length"],
    axis=1
)

first_model_df["TransactionsMediumBLength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_MediumB"] * row["Length"],
    axis=1
)

first_model_df["TransactionsHeavyALength"] = first_model_df.apply(
    lambda row: row["TransactionsDay_HeavyA"] * row["Length"],
    axis=1
)

first_model_df["TotalVeh_hours"] = first_model_df.apply(
    lambda row: row["Hours/Day"] * row["TotalVeh"],
    axis=1
)

first_model_df["GPDayLength_Lights"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_Lights"],
    axis=1
)

first_model_df["GPDayLength_MediumA"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_MediumA"],
    axis=1
)

first_model_df["GPDayLength_MediumB"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_MediumB"],
    axis=1
)

first_model_df["GPDayLength_HeavyA"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_HeavyA"],
    axis=1
)

first_model_df["GPDayLength_HeavyB"] = first_model_df.apply(
    lambda row: row["Length"] * row["GPDay_HeavyB"],
    axis=1
)

# Extra ones for the pivot

first_model_df["Difference in Travel Time"] = first_model_df.apply(
    lambda row: row["Time Savings"] / row["Time GP"],
    axis=1
)

first_model_df["Implied Min VOT"] = first_model_df.apply(
    lambda row: 60 * row["TollLightsPerSeg"] / (row["Time GP"] - row["Time ML"]),
    axis=1
)

# first_model_df.to_csv(f'I24_study/{today_str}/model_run.csv')

first_model_df.to_csv(f'{run_folder}/model_run.csv')

first_model_df

,Unnamed: 0,index,Year,SegDir,Segment,Direction,Period,Hours/Day,Peak,4Periods,...,TransactionsHeavyALength,TotalVeh_hours,GPDayLength_Lights,GPDayLength_MediumA,GPDayLength_MediumB,GPDayLength_HeavyA,GPDayLength_HeavyB,Difference in Travel Time,Implied Min VOT,Previous Seg
0,0,0,2025,1NB,1,NB,Night,7,OP,NT,...,21.510637,12054.000000,12445.123388,333.271504,90.078683,2235.289363,273.000000,-0.083882,131.058601,[]
1,1,1,2025,1NB,1,NB,AM-Early,1,OP,AM,...,16.460114,5777.000000,6057.245003,185.201770,96.893038,317.639886,15.600000,-0.116687,361.953811,[]
2,2,2,2025,1NB,1,NB,AM-Peak,3,Peak,AM,...,202.796364,20535.511652,18446.524846,810.834443,200.805174,1661.231977,43.148804,-0.381529,154.208771,[]
3,3,3,2025,1NB,1,NB,AM-Shoulder,1,OP,AM,...,83.456890,6282.794226,5478.465611,271.942362,75.473147,727.753832,24.376950,-0.353598,137.596336,[]
4,4,4,2025,1NB,1,NB,MD,5,OP,MD,...,330.848489,21465.000000,17967.727602,730.089949,199.114977,2139.151511,71.500000,-0.237947,153.493932,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,315,315,2032,10SB,10,SB,AM-Shoulder,1,OP,AM,...,188.029110,4121.101809,12452.244841,665.792801,289.599103,2107.584641,84.172504,-0.045132,1184.683855,[]
316,316,316,2032,10SB,10,SB,MD,5,OP,MD,...,758.239406,27933.000000,92341.140585,3652.293435,1270.182798,10829.260594,364.500000,-0.042698,1255.285180,[]
317,317,317,2032,10SB,10,SB,PM-Shoulder,1,OP,PM,...,80.177513,5023.000000,16988.583431,514.012773,128.502891,1061.922487,50.400000,-0.050432,1054.096237,[]
318,318,318,2032,10SB,10,SB,PM-Peak,3,Peak,PM,...,534.822620,16034.335131,49391.516944,1252.438686,238.497464,4595.928662,236.866735,-0.055850,943.932887,[]


In [241]:
period_order = [
    "Night",
    "AM-Early",
    "AM-Peak",
    "AM-Shoulder",
    "MD",
    "PM-Shoulder",
    "PM-Peak",
    "PM-Late"
]

first_model_df["Period"] = pd.Categorical(
    first_model_df["Period"],
    categories=period_order,
    ordered=True
)

first_model_df["GP V/C normal"] = first_model_df.apply(
    lambda row: row["GP PCE"] / seg_params.loc[row["SegDir"], 'Cap_GP'],
    axis=1
)

first_model_df["GP V/C capacity factors"] = first_model_df.apply(
    lambda row: row["GP PCE"] / row["Capacity GP"],
    axis=1
)

first_model_df["CaptureRateLights"] = first_model_df.apply(
    lambda row: row["CaptureRateLights"] * seg_params.loc[row["SegDir"], 'Inscope'],
    axis=1
)

C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\29839758.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  first_model_df["GP V/C normal"] = first_model_df.apply(
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\29839758.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  first_model_df["GP V/C capacity factors"] = first_model_df.apply(


In [242]:
# Define a light green style
def apply_light_green_style(df):
    return (df.style
    .set_table_styles([{
        'selector': 'th',
        'props': [
            ('background-color', "#7afc7f"),  # Light green
            ('color', '000000'),
            ('font-weight', 'bold'),
            ('border', '1px solid black')
        ]
    }])
    .set_properties(**{'border': '1px solid black'})  # Optional: add borders to data cells
)

In [243]:
# model_file = 'outputs/2025-12-01/best_current/model_run.csv'

# first_model_df = pd.read_csv(model_file)

# First Column

def generate_pivot(df, pivot_file, years):

    with pd.ExcelWriter(pivot_file, engine='openpyxl') as writer:

        for year_val in years:

            df_filtered = df[df["Year"] == year_val]

            lengths = df_filtered.pivot_table(
                index="Period",  # Single row index
                columns=["Direction", "SegDir"],
                values="Length",
                aggfunc='first'  # Takes the first value (since they're all the same)
            ).round(2)

            lengths = df_filtered[['Segment', 'Length']].drop_duplicates().set_index("Segment")

            lengths = lengths.T

            sumTollLights = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TollLights"
            ).round(2)

            sumCaptureRateLights = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="CaptureRateLights"
            ).round(2)

            revenuePerPeriod = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDay"
            ).round(2)

            revenuePerPeriod_LV = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayLights"
            ).round(2)

            revenuePerPeriod_MediumA = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayMediumA"
            ).round(2)

            revenuePerPeriod_MediumB = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayMediumB"
            ).round(2)

            revenuePerPeriod_HeavyA = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="RevenuePerDayHeavyA"
            ).round(2)

            percentage_revenue = revenuePerPeriod / revenuePerPeriod.sum()

            # Column 2

            ML_PCE_miles = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML PCE Total"
            ).round(2)

            ML_Volume = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML Volume"
            ).round(2)

            ML_flow = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML PCE"
            ).round(2)

            ML_vc = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="ML V/C"
            ).round(2)

            speed_ML = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Speed ML Real"
            ).round(2)

            ML_light_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_Lights"
            ).round(2)

            ML_MediumA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_MediumA"
            ).round(2)

            ML_MediumB_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_MediumB"
            ).round(2)

            ML_HeavyA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsDay_HeavyA"
            ).round(2)

            ML_light_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsLightsLength"
            ).round(2)

            ML_MediumA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsMediumALength"
            ).round(2)

            ML_MediumB_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsMediumBLength"
            ).round(2)

            ML_HeavyA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="TransactionsHeavyALength"
            ).round(2)

            time_ML = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time ML"
            ).round(2)

            time_savings_vot = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time Savings %"
            ).round(2)

            reliability = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Reliability %"
            ).round(2)

            bonus_vot = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Bonus %"
            ).round(2)

            bonus_rel = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="BonusRel"
            ).round(2)

            bonus_aux = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Bonus Seg"
            ).round(2)

            # Column 3

            GP_PCE_miles = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP PCE Total"
            ).round(2)

            GP_Volume = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP Volume"
            ).round(2)

            GP_flow = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP PCE"
            ).round(2)

            sum_GP_VC_nominal = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP V/C normal"
            ).round(2)

            sum_GP_VC = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GP V/C capacity factors"
            ).round(2)

            speed_GP = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Speed GP Real"
            ).round(2)

            time_GP = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time GP"
            ).round(2)

            GP_light_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_Lights"
            ).round(2)

            GP_MediumA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_MediumA"
            ).round(2)

            GP_MediumB_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_MediumB"
            ).round(2)

            GP_HeavyA_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_HeavyA"
            ).round(2)

            GP_HeavyB_veh_period = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDay_HeavyB"
            ).round(2)

            GP_light_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDayLength_Lights"
            ).round(2)

            GP_MediumA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDayLength_MediumA"
            ).round(2)

            GP_MediumB_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDayLength_MediumB"
            ).round(2)

            GP_HeavyA_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDayLength_HeavyA"
            ).round(2)

            GP_HeavyB_veh_mile = df_filtered.pivot_table(
                index="Period",
                columns=["Direction", "Segment"],
                values="GPDayLength_HeavyB"
            ).round(2)

            # Column 4

            corridor_PCE_total = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway PCE Total"
            ).round(2)

            corridor_volume = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway Volume"
            ).round(2)

            corridor_flow = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway PCE"
            ).round(2)

            corridor_vc = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Highway V/C"
            ).round(2)

            time_savings = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Time Savings"
            ).round(2)

            difference_time = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Difference in Travel Time"
            ).round(2)

            implied_vot = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="Implied Min VOT"
            ).round(2)

            toll_lights_seg = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TollLightsPerSeg"
            ).round(2)

            toll_mediumA_seg = df_filtered.pivot(
                index="Period",
                columns=["Direction", "Segment"],
                values="TollMediumAPerSeg"
            ).round(2)

            # sum_GPVeh = df_filtered.pivot(
            #     index="Period",
            #     columns=["Direction", "Segment"],
            #     values="GPVeh"
            # )

            # sum_MLVeh = df_filtered.pivot(
            #     index="Period",
            #     columns=["Direction", "Segment"],
            #     values="MLVeh"
            # )

            # sumRevHour = df_filtered.pivot(
            #     index="Period",
            #     columns=["Direction", "Segment"],
            #     values="RevenuePerHour"
            # )

            pivot_vals_column_1 = [sumTollLights, sumCaptureRateLights, revenuePerPeriod, revenuePerPeriod_LV, revenuePerPeriod_MediumA, revenuePerPeriod_MediumB, revenuePerPeriod_HeavyA, percentage_revenue]

            keys_1 = ["TollLights", "CaptureRateLights", "Revenue Per Period", "Revenue Per Period Lights", "Revenue Per Period Medium A", "Revenue Per Period Medium B", "Revenue Per Period Heavy A", "% Workday Revenue"]
            
            pivot_vals_1 = dict(zip(keys_1, pivot_vals_column_1))

            pivot_vals_column_2 = [ML_PCE_miles, ML_Volume, ML_flow, ML_vc, speed_ML, time_ML, ML_light_veh_mile, ML_light_veh_period, ML_MediumA_veh_mile, ML_MediumA_veh_period, ML_MediumB_veh_mile, ML_MediumB_veh_period, ML_HeavyA_veh_mile, ML_HeavyA_veh_period, time_savings_vot, bonus_rel, bonus_aux]
            
            keys_2 = ["ML PCE.Miles", "ML Volume (PCE/Period)", "ML Flow (PCE/hr)", "ML V/C", "ML Speed", "ML time", "ML light veh.Miles", "ML light veh/period", "ML medium A veh.Miles", "ML medium A veh/period", "ML medium B veh.Miles", "ML medium B veh/period", "ML heavy A veh.Miles", "ML heavy A veh/period", "Reliability", "TB"]

            pivot_vals_2 = dict(zip(keys_2, pivot_vals_column_2))
            
            pivot_vals_column_3 = [GP_PCE_miles, GP_Volume, GP_flow, sum_GP_VC, speed_GP, time_GP, GP_light_veh_mile, GP_light_veh_period, GP_MediumA_veh_mile, GP_MediumA_veh_period, GP_MediumB_veh_mile, GP_MediumB_veh_period, GP_HeavyA_veh_mile, GP_HeavyA_veh_period, GP_HeavyB_veh_mile, GP_HeavyB_veh_period]

            keys_3 = ["GP PCE.Miles", "GP Volume (PCE/Period)", "GP Flow (PCE/hr)", "GP V/C real", "GP Speed", "GP Time", "GP light veh.Miles", "GP light veh/period", "GP medium A veh.Miles", "GP medium A veh/period", "GP medium B veh.Miles", "GP medium B veh/period", "GP heavy A veh.Miles", "GP heavy A veh/period", "GP heavy B veh.Miles", "GP heavy B veh/period"]
            
            pivot_vals_3 = dict(zip(keys_3, pivot_vals_column_3))

            pivot_vals_column_4 = [corridor_PCE_total, corridor_volume, corridor_flow, corridor_vc, time_savings, difference_time, toll_lights_seg, toll_mediumA_seg, implied_vot]

            keys_4 = ["Highway PCE.Miles", "Highway Volume", "Highway Flow", "Highway Vol / GP Capacity", "Time Saving", "Difference in travel time", "LVr Toll", "Medium A Toll", "Implied VOT"]

            pivot_vals_4 = dict(zip(keys_4, pivot_vals_column_4))

            pd.DataFrame().to_excel(writer, sheet_name=f'Year_{year_val}')

            startcol = 1

            start_row = 0

            worksheet = writer.sheets[f'Year_{year_val}']

            worksheet.cell(start_row + 1, startcol + 1, "Lenght")

            styled_table = lengths

            # styled_table = styled_table.format("{:.2f}")

            styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row + 1, startcol=startcol)

            start_row += lengths.shape[0] + 3

            for key, pivot_vals in pivot_vals_1.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1] - 1

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

            start_row = 4
        
            for key, pivot_vals in pivot_vals_2.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1]

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

            start_row = 4

            for key, pivot_vals in pivot_vals_3.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1]

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

            start_row = 4

            for key, pivot_vals in pivot_vals_4.items():

                worksheet = writer.sheets[f'Year_{year_val}']

                worksheet.cell(start_row + 1, startcol + 1, key)

                start_row += 1

                start_column_cumm = pivot_vals.shape[1]

                styled_table = apply_light_green_style(pivot_vals)

                styled_table = styled_table.format("{:.2f}")

                styled_table.to_excel(writer, sheet_name=f'Year_{year_val}', startrow=start_row, startcol=startcol)

                start_row += pivot_vals.shape[0] + 5
            
            startcol += start_column_cumm + 3

In [244]:
years = [2025,2032,2040,2050]

generate_pivot(first_model_df, f'{run_folder}/pivot_consecutive.xlsx', years)

C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\2945143176.py:15: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  lengths = df_filtered.pivot_table(
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\2945143176.py:126: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  ML_light_veh_mile = df_filtered.pivot_table(
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\2945143176.py:132: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  ML_MediumA_veh_mile = df_filtered.pivot_table(
C:\Users\crisp\AppData\Local\Temp\ipy

In [245]:
def generate_revenue_stream(final_df, file_name):

    output_df = pd.DataFrame(columns=['Year', 'Total Revenue', 'Base Revenues', 'Dev Revenues', 'Dev Revenues 2x', 'Dev Revenues 3x', 'Dev Revenues 4x', 'TDOT Revenues', 'TDOT Revenues 2x', 'TDOT Revenues 3x', 'TDOT Revenues 4x', 'Total Transactions', 'Transactions Lights', 'Transactions Medium A', 'Transactions Medium B', 'Transactions Heavy A', 'AADT', 'Toll/AADT/mi', 'Toll Lights', 'Toll Medium A', 'Toll Meadium B', 'Toll Heavy A', 'Capture', 'Corridor PCE']) # AADT refers to ML

    years = [2025, 2032, 2040, 2050]

    Anualization_factor = 296 # TBD: Just changed from 290, check if it makes sense
    traffic_anualization = 310 # TBD: Check if it fits
    year_days = 365

    length_correction_factor = 1

    for year in years:

        yearly_df = final_df[final_df["Year"] == year]

        # We compute the revenues

        revenues_df = yearly_df.groupby(['Segment', 'Direction'])[["RevenuePerDay"]].sum().reset_index() #

        revenues_df_lights = yearly_df.groupby(['Segment', 'Direction'])[["RevenuePerDayLights"]].sum().reset_index() #

        revenues_dev = yearly_df.groupby(['Segment', 'Direction'])[["DevRevenuePerDay"]].sum().reset_index()

        revenues_dev2x = yearly_df.groupby(['Segment', 'Direction'])[["DevRevenuePerDay2x"]].sum().reset_index()
        revenues_dev3x = yearly_df.groupby(['Segment', 'Direction'])[["DevRevenuePerDay3x"]].sum().reset_index()
        revenues_dev4x = yearly_df.groupby(['Segment', 'Direction'])[["DevRevenuePerDay4x"]].sum().reset_index()

        revenues_tdot = yearly_df.groupby(['Segment', 'Direction'])[["TDOTRevenuePerDay"]].sum().reset_index()

        revenues_tdot2x = yearly_df.groupby(['Segment', 'Direction'])[["TDOTRevenuePerDay2x"]].sum().reset_index()
        revenues_tdot3x = yearly_df.groupby(['Segment', 'Direction'])[["TDOTRevenuePerDay3x"]].sum().reset_index()
        revenues_tdot4x = yearly_df.groupby(['Segment', 'Direction'])[["TDOTRevenuePerDay4x"]].sum().reset_index()

        revenues_base = yearly_df.groupby(['Segment', 'Direction'])[["BaseRevenuePerDay"]].sum().reset_index()

        revenues_df["Annual Revenue"] = revenues_df["RevenuePerDay"] * Anualization_factor # We have to change 290 for an annual constant

        revenues_df_lights["Annual Revenue"] = revenues_df_lights["RevenuePerDayLights"] * Anualization_factor # We have to change 290 for an annual constant

        revenues_dev["Annual Revenue"] = revenues_dev["DevRevenuePerDay"] * Anualization_factor

        revenues_dev2x["Annual Revenue"] = revenues_dev2x["DevRevenuePerDay2x"] * Anualization_factor
        revenues_dev3x["Annual Revenue"] = revenues_dev3x["DevRevenuePerDay3x"] * Anualization_factor
        revenues_dev4x["Annual Revenue"] = revenues_dev4x["DevRevenuePerDay4x"] * Anualization_factor

        revenues_tdot["Annual Revenue"] = revenues_tdot["TDOTRevenuePerDay"] * Anualization_factor

        revenues_tdot2x["Annual Revenue"] = revenues_tdot2x["TDOTRevenuePerDay2x"] * Anualization_factor
        revenues_tdot3x["Annual Revenue"] = revenues_tdot3x["TDOTRevenuePerDay3x"] * Anualization_factor
        revenues_tdot4x["Annual Revenue"] = revenues_tdot4x["TDOTRevenuePerDay4x"] * Anualization_factor

        revenues_base["Annual Revenue"] = revenues_base["BaseRevenuePerDay"] * Anualization_factor

        total_revenues = revenues_df["Annual Revenue"].sum() * length_correction_factor

        total_revenues_lights = revenues_df_lights["Annual Revenue"].sum() * length_correction_factor

        total_revenues_dev = revenues_dev["Annual Revenue"].sum() * length_correction_factor

        total_revenues_dev2x = revenues_dev2x["Annual Revenue"].sum() * length_correction_factor
        total_revenues_dev3x = revenues_dev3x["Annual Revenue"].sum() * length_correction_factor
        total_revenues_dev4x = revenues_dev4x["Annual Revenue"].sum() * length_correction_factor

        total_revenues_tdot = revenues_tdot["Annual Revenue"].sum() * length_correction_factor

        total_revenues_tdot2x = revenues_tdot2x["Annual Revenue"].sum() * length_correction_factor
        total_revenues_tdot3x = revenues_tdot3x["Annual Revenue"].sum() * length_correction_factor
        total_revenues_tdot4x = revenues_tdot4x["Annual Revenue"].sum() * length_correction_factor

        total_revenues_base = revenues_base["Annual Revenue"].sum() * length_correction_factor

        # We compute the total transactions

        transactions_df = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay"]].sum().reset_index()

        transactions_df_lights = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay_Lights"]].sum().reset_index()
        transactions_df_medium_A = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay_MediumA"]].sum().reset_index()
        transactions_df_medium_B = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay_MediumB"]].sum().reset_index()
        transactions_df_heavy_A = yearly_df.groupby(['Segment', 'Direction'])[["TransactionsDay_HeavyA"]].sum().reset_index()


        # transactions_df_aux.append(transactions_df)

        total_transactions = transactions_df["TransactionsDay"].sum() * traffic_anualization

        transactions_lights = transactions_df_lights["TransactionsDay_Lights"].sum() * traffic_anualization
        transactions_mediumA = transactions_df_medium_A["TransactionsDay_MediumA"].sum() * traffic_anualization
        transactions_mediumB = transactions_df_medium_B["TransactionsDay_MediumB"].sum() * traffic_anualization
        transactions_heavyA = transactions_df_heavy_A["TransactionsDay_HeavyA"].sum() * traffic_anualization

        # We compute the AADT

        aadt_df = yearly_df.groupby(['SegDir'])[["TransactionsDay"]].sum()

        aadt_df_lights = yearly_df.groupby(['SegDir'])[["TransactionsDay_Lights"]].sum()

        gp_traffic_df = yearly_df.groupby(['SegDir'])[["GPVehDay"]].sum()

        corridor_df = yearly_df.groupby(['SegDir'])[["Highway Volume"]].sum()

        total_df = yearly_df.groupby(['SegDir'])[["TotalVeh_hours"]].sum()

        df_lengths = pd.DataFrame(seg_params["Length"])

        merged_df = pd.merge(aadt_df, df_lengths, on='SegDir')

        merged_df_lights = pd.merge(aadt_df_lights, df_lengths, on='SegDir')

        merged_gp_traffic_df = pd.merge(gp_traffic_df, df_lengths, on='SegDir')

        merged_corridor_df = pd.merge(corridor_df, df_lengths, on='SegDir')

        # Compute product
        merged_df['Weighted'] = merged_df['Length'].values * merged_df["TransactionsDay"].values

        merged_df_lights['Weighted'] = merged_df_lights['Length'].values * merged_df_lights["TransactionsDay_Lights"].values

        merged_gp_traffic_df['Weighted'] = merged_gp_traffic_df['Length'] * merged_gp_traffic_df["GPVehDay"]

        merged_corridor_df['Weighted'] = merged_corridor_df['Length'] * merged_corridor_df["Highway Volume"]

        # Sum weighted values
        total_aadt = (traffic_anualization/year_days) * merged_df['Weighted'].sum() / (merged_df['Length'].sum()/2)

        total_aadt_lights = (traffic_anualization/year_days) * merged_df_lights['Weighted'].sum() / (merged_df_lights['Length'].sum()/2)        

        traffic_gp = (traffic_anualization/year_days) * merged_gp_traffic_df['Weighted'].sum() / (merged_gp_traffic_df['Length'].sum()/2)

        gp_ml = total_aadt + traffic_gp

        corridor_pce = (traffic_anualization/year_days) * merged_corridor_df['Weighted'].sum() / (merged_corridor_df['Length'].sum()/2)

        capture_total = total_aadt / gp_ml

        toll_AADT_mi = total_revenues / (year_days * total_aadt * (merged_df['Length'].sum()/2))

        toll_AADT_lights = total_revenues_lights / (year_days * total_aadt_lights * (merged_df['Length'].sum()/2))

        toll_AADT_mediumA = toll_AADT_lights * medium_A_w_toll

        toll_AADT_mediumB = toll_AADT_lights * medium_B_w_toll

        toll_AADT_heavyA = toll_AADT_lights * heavy_A_w_toll
        
        output_df.loc[len(output_df)] = [year, total_revenues, total_revenues_base, total_revenues_dev, total_revenues_dev2x, total_revenues_dev3x, total_revenues_dev4x, total_revenues_tdot, total_revenues_tdot2x, total_revenues_tdot3x, total_revenues_tdot4x, total_transactions, transactions_lights, transactions_mediumA, transactions_mediumB, transactions_heavyA, total_aadt, toll_AADT_mi, toll_AADT_lights, toll_AADT_mediumA, toll_AADT_mediumB, toll_AADT_heavyA, capture_total, corridor_pce]

    # Define full range of years to interpolate over
    full_years = pd.DataFrame({"Year": np.arange(int(output_df["Year"].min()), int(output_df["Year"].max()) + 1)})

    # Merge with original to create missing years with NaNs
    merged = pd.merge(full_years, output_df, on="Year", how="left")

    # Interpolate all numeric columns except "Year"
    interpolated = np.log(merged).interpolate(method="linear")

    interpolated = np.exp(interpolated)

    # Copy original column names
    original_columns = list(interpolated.columns)

    # Track how many columns we've inserted to adjust the index
    insert_count = 0

    # Start from index 1 to skip the first column (index 0)
    for i in range(1, len(original_columns)):
        col = original_columns[i]
        new_col_name = f"Var {col} (%)"
        insert_position = i + insert_count + 1  # Adjust position with insert_count
        interpolated[col] = interpolated[col].round(3)
        new_col_val = interpolated[col].pct_change().round(3)
        interpolated.insert(insert_position, new_col_name, new_col_val)
        insert_count += 1

    interpolated.to_csv(file_name)

    return interpolated

In [246]:
extensions_segdir_NE = ['11NB','12NB','18NB'] + ['11SB','12SB','18SB']

extensions_segdir_EE = ['13NB','14NB'] + ['13SB','14SB']

extensions_segdir_SE = ['15NB','16NB','17NB'] + ['15SB','16SB','17SB']

extensions_NE = first_model_df[first_model_df['SegDir'].isin(extensions_segdir_NE)]

extensions_EE = first_model_df[first_model_df['SegDir'].isin(extensions_segdir_EE)]

extensions_SE = first_model_df[first_model_df['SegDir'].isin(extensions_segdir_SE)]

# revenue_stream_subphase_12 = generate_revenue_stream(subphase_12, f'{run_folder}/revenue_stream_phase12.csv')

generate_revenue_stream_extensions_NE = generate_revenue_stream(extensions_NE, f'consecutive_increase/revenue_stream_extensions_NE.csv')

generate_revenue_stream_extensions_EE = generate_revenue_stream(extensions_EE, f'consecutive_increase/revenue_stream_extensions_EE.csv')

generate_revenue_stream_extensions_SE = generate_revenue_stream(extensions_SE, f'consecutive_increase/revenue_stream_extensions_SE.csv')

interpolated = generate_revenue_stream(first_model_df, f'consecutive_increase/revenue_stream.csv')

interpolated

C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\222033933.py:124: RuntimeWarning: invalid value encountered in scalar divide
  total_aadt = (traffic_anualization/year_days) * merged_df['Weighted'].sum() / (merged_df['Length'].sum()/2)
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\222033933.py:126: RuntimeWarning: invalid value encountered in scalar divide
  total_aadt_lights = (traffic_anualization/year_days) * merged_df_lights['Weighted'].sum() / (merged_df_lights['Length'].sum()/2)
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\222033933.py:128: RuntimeWarning: invalid value encountered in scalar divide
  traffic_gp = (traffic_anualization/year_days) * merged_gp_traffic_df['Weighted'].sum() / (merged_gp_traffic_df['Length'].sum()/2)
C:\Users\crisp\AppData\Local\Temp\ipykernel_26328\222033933.py:132: RuntimeWarning: invalid value encountered in scalar divide
  corridor_pce = (traffic_anualization/year_days) * merged_corridor_df['Weighted'].sum() / (merged_corridor_df['Length'

,Year,Total Revenue,Var Total Revenue (%),Base Revenues,Var Base Revenues (%),Dev Revenues,Var Dev Revenues (%),Dev Revenues 2x,Var Dev Revenues 2x (%),Dev Revenues 3x,...,Toll Medium A,Var Toll Medium A (%),Toll Meadium B,Var Toll Meadium B (%),Toll Heavy A,Var Toll Heavy A (%),Capture,Var Capture (%),Corridor PCE,Var Corridor PCE (%)
0,2025.0,2.168461e+08,NaN,2.168461e+08,NaN,0.000,NaN,0.000,NaN,0.0,...,3.657,NaN,3.657,NaN,6.401,NaN,0.141,NaN,183079.804,NaN
1,2026.0,2.419914e+08,0.116,2.419611e+08,0.116,0.000,NaN,0.000,NaN,0.0,...,3.797,0.038,3.797,0.038,6.645,0.038,0.147,0.043,188316.189,0.029
2,2027.0,2.700527e+08,0.116,2.699850e+08,0.116,0.000,NaN,0.000,NaN,0.0,...,3.942,0.038,3.942,0.038,6.899,0.038,0.154,0.048,193702.344,0.029
3,2028.0,3.013679e+08,0.116,3.012545e+08,0.116,0.000,NaN,0.000,NaN,0.0,...,4.093,0.038,4.093,0.038,7.163,0.038,0.161,0.045,199242.552,0.029
4,2029.0,3.363144e+08,0.116,3.361457e+08,0.116,0.000,NaN,0.000,NaN,0.0,...,4.249,0.038,4.249,0.038,7.436,0.038,0.168,0.043,204941.218,0.029
5,2030.0,3.753133e+08,0.116,3.750780e+08,0.116,0.000,NaN,0.000,NaN,0.0,...,4.412,0.038,4.412,0.038,7.721,0.038,0.175,0.042,210802.877,0.029
6,2031.0,4.188345e+08,0.116,4.185195e+08,0.116,0.000,NaN,0.000,NaN,0.0,...,4.580,0.038,4.580,0.038,8.016,0.038,0.183,0.046,216832.188,0.029
7,2032.0,4.674023e+08,0.116,4.669923e+08,0.116,205037.565,inf,205037.565,inf,0.0,...,4.755,0.038,4.755,0.038,8.322,0.038,0.192,0.049,223033.947,0.029
8,2033.0,0.000000e+00,-1.000,0.000000e+00,-1.000,0.000,-1.0,0.000,-1.0,0.0,...,4.755,0.000,4.755,0.000,8.322,0.000,0.192,0.000,223033.947,0.000
9,2034.0,0.000000e+00,NaN,0.000000e+00,NaN,0.000,NaN,0.000,NaN,0.0,...,4.755,0.000,4.755,0.000,8.322,0.000,0.192,0.000,223033.947,0.000
